In [5]:
# prompt: update satasets class

!pip install datasets -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 21.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [2]:
import torch
import math
import numpy as np
from datasets import load_dataset
from transformers import (
    GPT2LMHeadModel,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
)

# Set device explicitly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Using device: {device}")

# 1. Load 5% of WikiText-2
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
dataset["train"] = dataset["train"].select(range(int(0.50 * len(dataset["train"]))))
dataset["validation"] = dataset["validation"].select(range(int(0.5 * len(dataset["validation"]))))
dataset["test"] = dataset["test"].select(range(int(0.5 * len(dataset["test"]))))

# 2. Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# 3. Tokenize
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=["text"])
tokenized_dataset.set_format("torch")

# 4. Load model
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.eos_token_id
model.to(device)

# 5. Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 6. Training args
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    logging_steps=10,
    fp16=True,  # GPU acceleration with mixed precision
    report_to="none",
    push_to_hub=False
)

# 7. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# 8. Fine-tune
trainer.train()

# 9. Evaluate: compute loss & perplexity
eval_results = trainer.evaluate(tokenized_dataset["test"])
perplexity = math.exp(eval_results["eval_loss"])
print(f"\n📉 Perplexity: {perplexity:.2f}")

# 10. (Optional) Top-k accuracy function
def compute_top_k_accuracy(model, dataset, k=5, num_batches=10):
    model.eval()
    correct = 0
    total = 0

    dataloader = torch.utils.data.DataLoader(dataset, batch_size=1)

    with torch.no_grad():
        for idx, batch in enumerate(dataloader):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)

            logits = outputs.logits
            next_token_logits = logits[:, -2, :]  # Predict next word
            top_k_preds = torch.topk(next_token_logits, k, dim=-1).indices

            true_token = input_ids[0, -1]
            if true_token in top_k_preds[0]:
                correct += 1
            total += 1

            if idx >= num_batches:
                break

    return correct / total if total > 0 else 0

topk_acc = compute_top_k_accuracy(model, tokenized_dataset["test"], k=5, num_batches=100)
print(f"🎯 Top-5 Accuracy: {topk_acc:.2%}")


🚀 Using device: cuda


Map:   0%|          | 0/2179 [00:00<?, ? examples/s]

Map:   0%|          | 0/18359 [00:00<?, ? examples/s]

Map:   0%|          | 0/1880 [00:00<?, ? examples/s]

/tmp/ipython-input-2-3875980188.py:56: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,3.910400
20,4.017500
30,3.866500
40,3.992100
50,3.855600
60,3.847600
70,3.764600
80,3.829300
90,3.748500
100,3.731400


Step,Training Loss
10,3.910400
20,4.017500
30,3.866500
40,3.992100
50,3.855600
60,3.847600
70,3.764600
80,3.829300
90,3.748500
100,3.731400



📉 Perplexity: 29.20
🎯 Top-5 Accuracy: 11.88%
